# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d1e0fd1f74-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

## Task 1
Identify the BGP for Cultural Movement

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name ?oName
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oName
}
ORDER BY ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9493'), ('name', 'artist files at'), ('oName', 'Frick Art Reference Library')]
[('p', 'http://www.wikidata.org/prop/direct/P9493'), ('name', 'artist files at'), ('oName', 'Berenson Library')]
[('p', 'http://www.wikidata.org/prop/direct/P7763'), ('name', 'copyright status as a creator'), ('oName', 'copyrights on works have expired')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship'), ('oName', 'Republic of Florence')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('oName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('oName', 'Donatello

In [3]:
# donatello movement
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q37562 wdt:P135 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
1


In [4]:
# Early renaissance instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q37562 wdt:P135 ?obj .
   ?obj wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period')]
[('o', 'http://www.wikidata.org/entity/Q968159'), ('name', 'art movement')]
2


In [5]:
# all props of Early renaissance
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
  
   wd:Q1472236 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q968159'), ('name', 'art movement')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q979160'), ('name', 'Proto-Renaissance')]
5


In [6]:
# all props of art movement
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
  
   wd:Q968159 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7458526'), ('name', 'Category:Art movements')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q735'), ('name', 'art')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q4502119'), ('name', 'art group')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q1293970'), ('name', 'list of art movements')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q14412130'), ('name', 'Template:Infobox art movement')]
6


In [7]:
# all props of cultural movement
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
  
   wd:Q2198855 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q49773'), ('name', 'social movement')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3533467'), ('name', 'group action')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6638204'), ('name', 'Category:Cultural movement')]
4


Final query for this task

In [8]:
# BGP for Cultural Movement
queryString = """
SELECT DISTINCT ?iri ?name
WHERE {
  
   BIND(wd:Q2198855 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
1


In [9]:
# all props of art movement
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
  
   wd:Q968159 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7458526'), ('name', 'Category:Art movements')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q735'), ('name', 'art')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q4502119'), ('name', 'art group')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q1293970'), ('name', 'list of art movements')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q14412130'), ('name', 'Template:Infobox art movement')]
6


## Task 2
Identify the BGP for Renassaince, Baroque and Romanticism

In [10]:
# all instances of art movement
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
  
   ?s wdt:P31 wd:Q968159;
       sc:name ?name.
}
ORDER BY ?name
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q25347092'), ('name', 'Accidentalism')]
[('s', 'http://www.wikidata.org/entity/Q556997'), ('name', 'Activism')]
[('s', 'http://www.wikidata.org/entity/Q10401752'), ('name', 'Aemilia Ars')]
[('s', 'http://www.wikidata.org/entity/Q38157'), ('name', 'African studies')]
[('s', 'http://www.wikidata.org/entity/Q2828843'), ('name', 'Aizuri-e')]
[('s', 'http://www.wikidata.org/entity/Q2828892'), ('name', 'Aka-e')]
[('s', 'http://www.wikidata.org/entity/Q1000039'), ('name', 'Akita ranga')]
[('s', 'http://www.wikidata.org/entity/Q466675'), ('name', 'American Renaissance')]
[('s', 'http://www.wikidata.org/entity/Q4745501'), ('name', 'American modernism')]
[('s', 'http://www.wikidata.org/entity/Q4745527'), ('name', 'American realism')]
[('s', 'http://www.wikidata.org/entity/Q469913'), ('name', 'American scene painting')]
[('s', 'http://www.wikidata.org/entity/Q478717'), ('name', 'Amsterdam Impressionism')]
[('s', 'http://www.wikidata.org/entity/Q30833

In [11]:
# all prev and after of early renaissance
queryString = """
SELECT DISTINCT ?n ?name
WHERE {
  
   wd:Q1472236 wdt:P155+|wdt:P156+ ?n. #prev
   ?n sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('n', 'http://www.wikidata.org/entity/Q979160'), ('name', 'Proto-Renaissance')]
[('n', 'http://www.wikidata.org/entity/Q131808'), ('name', 'Mannerism')]
[('n', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance')]
[('n', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
4


Final query for this task

In [12]:
# BGP for Baroque, Renaissance and Romanticism
queryString = """
SELECT DISTINCT ?iri ?name
WHERE {
   VALUES ?iri {wd:Q37068 wd:Q4692 wd:Q37853}
   ?iri sc:name ?name.
}
ORDER BY ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('iri', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
[('iri', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
3


In [13]:
# all instances of art movement
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
  
   ?s wdt:P31 wd:Q968159;
       sc:name ?name.
}
ORDER BY ?name
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q25347092'), ('name', 'Accidentalism')]
[('s', 'http://www.wikidata.org/entity/Q556997'), ('name', 'Activism')]
[('s', 'http://www.wikidata.org/entity/Q10401752'), ('name', 'Aemilia Ars')]
[('s', 'http://www.wikidata.org/entity/Q38157'), ('name', 'African studies')]
[('s', 'http://www.wikidata.org/entity/Q2828843'), ('name', 'Aizuri-e')]
[('s', 'http://www.wikidata.org/entity/Q2828892'), ('name', 'Aka-e')]
[('s', 'http://www.wikidata.org/entity/Q1000039'), ('name', 'Akita ranga')]
[('s', 'http://www.wikidata.org/entity/Q466675'), ('name', 'American Renaissance')]
[('s', 'http://www.wikidata.org/entity/Q4745501'), ('name', 'American modernism')]
[('s', 'http://www.wikidata.org/entity/Q4745527'), ('name', 'American realism')]
[('s', 'http://www.wikidata.org/entity/Q469913'), ('name', 'American scene painting')]
[('s', 'http://www.wikidata.org/entity/Q478717'), ('name', 'Amsterdam Impressionism')]
[('s', 'http://www.wikidata.org/entity/Q30833

## Task 3
Identify the BGP for composer, painter, sculptor and poet

In [14]:
# donatello occupations
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q37562 wdt:P106 ?o.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('o', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
[('o', 'http://www.wikidata.org/entity/Q483501'), ('name', 'artist')]
3


In [15]:
# all painter props
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q1028181 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q1027879'), ('oName', 'graphics')]
[('p', 'http://www.wikidata.org/prop/direct/P417'), ('name', 'patron saint'), ('o', 'http://www.wikidata.org/entity/Q128538'), ('oName', 'Luke the Evangelist')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5914426'), ('oName', 'Template:Infobox artist')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6615720'), ('oName', 'Category:Painters')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q11629'), ('oName', 'art of painting')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q12737077'), ('oName', 'occupation')]
[('

In [16]:
# all instances of occupation
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?o wdt:P31 wd:Q12737077.
   ?o sc:name ?name.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q104215583'), ('name', 'ASMR artist')]
[('o', 'http://www.wikidata.org/entity/Q8380347'), ('name', 'AV actor')]
[('o', 'http://www.wikidata.org/entity/Q1079215'), ('name', 'AV idol')]
[('o', 'http://www.wikidata.org/entity/Q14161749'), ('name', 'Absolvent')]
[('o', 'http://www.wikidata.org/entity/Q17048963'), ('name', 'Advokat')]
[('o', 'http://www.wikidata.org/entity/Q20105472'), ('name', 'Agente Forestal')]
[('o', 'http://www.wikidata.org/entity/Q5659846'), ('name', 'Agente de Medio Ambiente de la Junta de Andalucía')]
[('o', 'http://www.wikidata.org/entity/Q66771409'), ('name', 'Aliyah Bet activist')]
[('o', 'http://www.wikidata.org/entity/Q106595070'), ('name', 'American Indian law scholar')]
[('o', 'http://www.wikidata.org/entity/Q42331263'), ('name', 'American football coach')]
[('o', 'http://www.wikidata.org/entity/Q2981922'), ('name', 'American pioneer')]
[('o', 'http://www.wikidata.org/entity/Q99899510'), ('name', 'Ancient Roman h

In [17]:
# all romantics
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P135 wd:Q37068.
       
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q16038489'), ('name', 'Cross and cathedral in the mountains')]
[('s', 'http://www.wikidata.org/entity/Q16038674'), ('name', 'Bohemian Landscape with Mount Milleschauer')]
[('s', 'http://www.wikidata.org/entity/Q1065323'), ('name', 'Charles Leickert')]
[('s', 'http://www.wikidata.org/entity/Q1375979'), ('name', 'Raden Saleh')]
[('s', 'http://www.wikidata.org/entity/Q512948'), ('name', 'Jan Willem Pieneman')]
[('s', 'http://www.wikidata.org/entity/Q576824'), ('name', 'Jan Adam Kruseman')]
[('s', 'http://www.wikidata.org/entity/Q2081067'), ('name', 'Petrus van Schendel')]
[('s', 'http://www.wikidata.org/entity/Q76416'), ('name', 'Andreas Achenbach')]
[('s', 'http://www.wikidata.org/entity/Q2958563'), ('name', 'Charles Bird King')]
[('s', 'http://www.wikidata.org/entity/Q5932129'), ('name', 'Joaquín José Cervino')]
[('s', 'http://www.wikidata.org/entity/Q3163756'), ('name', 'Ange Tissier')]
[('s', 'http://www.wikidata.org/entity/Q20973384'), (

In [18]:
# all romantics
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   ?s wdt:P135 wd:Q37068;
       wdt:P101 ?o.
   ?o sc:name ?name.
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q9730'), ('name', 'Western classical music')]
[('o', 'http://www.wikidata.org/entity/Q12271'), ('name', 'architecture')]
[('o', 'http://www.wikidata.org/entity/Q1583807'), ('name', 'art music')]
[('o', 'http://www.wikidata.org/entity/Q11629'), ('name', 'art of painting')]
[('o', 'http://www.wikidata.org/entity/Q11634'), ('name', 'art of sculpture')]
[('o', 'http://www.wikidata.org/entity/Q4914883'), ('name', 'biographical novel')]
[('o', 'http://www.wikidata.org/entity/Q186030'), ('name', 'contemporary art')]
[('o', 'http://www.wikidata.org/entity/Q17955'), ('name', 'criticism')]
[('o', 'http://www.wikidata.org/entity/Q492537'), ('name', 'dramaturgy')]
[('o', 'http://www.wikidata.org/entity/Q132151'), ('name', 'ethnography')]
[('o', 'http://www.wikidata.org/entity/Q699'), ('name', 'fairy tale')]
[('o', 'http://www.wikidata.org/entity/Q8253972'), ('name', 'fiction')]
[('o', 'http://www.wikidata.org/entity/Q132116'), ('name', 'folkloristics'

In [19]:
# all musical composition props
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
   wd:Q207628 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P407'), ('name', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('o', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P8952'), ('pName', 'inappropriate property for this type'), ('o', 'http://www.wikidata.org/entity/P264'), ('name', 'rec

In [20]:
# all poetry props
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
   wd:Q482 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pName', 'model item'), ('o', 'http://www.wikidata.org/entity/Q18146776'), ('name', 'Olympic Odes')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pName', 'model item'), ('o', 'http://www.wikidata.org/entity/Q27214182'), ('name', 'Nemean Odes')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pName', 'model item'), ('o', 'http://www.wikidata.org/entity/Q520372'), ('name', 'Lancelot, the Knight of the Cart')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8263'), ('name', 'Category:Poetry')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1768721'), ('name', 'G

In [21]:
# final
queryString = """
SELECT DISTINCT ?iri ?name
WHERE {
   VALUES ?iri {wd:Q49757 wd:Q36834 wd:Q1028181 wd:Q1281618}
   ?iri sc:name ?name.
}
ORDER BY ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('iri', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('iri', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('iri', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
4


In [22]:
# all poet props
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
   wd:Q49757 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q482'), ('name', 'poetry')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q1293108'), ('name', 'Dichter')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6487149'), ('name', 'Category:Poets')]
[('p', 'http://w

Final query for this task

In [23]:
# BGP for composer, painter, poet and sculptor
queryString = """
SELECT DISTINCT ?iri ?name
WHERE {
   VALUES ?iri {wd:Q49757 wd:Q36834 wd:Q1028181 wd:Q1281618}
   ?iri sc:name ?name.
}
ORDER BY ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('iri', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('iri', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('iri', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
4


## Task 4
Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

In [24]:
#final 
queryString = """
SELECT DISTINCT ?movement ?job (COUNT(?p) AS ?Npeople)
WHERE {
   VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
   VALUES ?j {wd:Q49757 wd:Q36834 wd:Q1028181 wd:Q1281618}
   ?p wdt:P106 ?j;
       wdt:P135 ?mov.
   ?mov sc:name ?movement.
   ?j sc:name ?job.
   
}
GROUP BY ?movement ?job
ORDER BY ?movement ?job
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'Baroque'), ('job', 'composer'), ('Npeople', '6')]
[('movement', 'Baroque'), ('job', 'painter'), ('Npeople', '378')]
[('movement', 'Baroque'), ('job', 'poet'), ('Npeople', '39')]
[('movement', 'Baroque'), ('job', 'sculptor'), ('Npeople', '328')]
[('movement', 'Renaissance'), ('job', 'composer'), ('Npeople', '8')]
[('movement', 'Renaissance'), ('job', 'painter'), ('Npeople', '56')]
[('movement', 'Renaissance'), ('job', 'poet'), ('Npeople', '15')]
[('movement', 'Renaissance'), ('job', 'sculptor'), ('Npeople', '18')]
[('movement', 'Romanticism'), ('job', 'composer'), ('Npeople', '13')]
[('movement', 'Romanticism'), ('job', 'painter'), ('Npeople', '152')]
[('movement', 'Romanticism'), ('job', 'poet'), ('Npeople', '151')]
[('movement', 'Romanticism'), ('job', 'sculptor'), ('Npeople', '14')]
12


Final query for this task

In [25]:
#final query
queryString = """
SELECT DISTINCT ?movement ?job (COUNT(?p) AS ?Npeople)
WHERE {
   VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
   VALUES ?j {wd:Q49757 wd:Q36834 wd:Q1028181 wd:Q1281618}
   ?p wdt:P106 ?j;
       wdt:P135 ?mov.
   ?mov sc:name ?movement.
   ?j sc:name ?job.
   
}
GROUP BY ?movement ?job
ORDER BY ?movement ?job
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'Baroque'), ('job', 'composer'), ('Npeople', '6')]
[('movement', 'Baroque'), ('job', 'painter'), ('Npeople', '378')]
[('movement', 'Baroque'), ('job', 'poet'), ('Npeople', '39')]
[('movement', 'Baroque'), ('job', 'sculptor'), ('Npeople', '328')]
[('movement', 'Renaissance'), ('job', 'composer'), ('Npeople', '8')]
[('movement', 'Renaissance'), ('job', 'painter'), ('Npeople', '56')]
[('movement', 'Renaissance'), ('job', 'poet'), ('Npeople', '15')]
[('movement', 'Renaissance'), ('job', 'sculptor'), ('Npeople', '18')]
[('movement', 'Romanticism'), ('job', 'composer'), ('Npeople', '13')]
[('movement', 'Romanticism'), ('job', 'painter'), ('Npeople', '152')]
[('movement', 'Romanticism'), ('job', 'poet'), ('Npeople', '151')]
[('movement', 'Romanticism'), ('job', 'sculptor'), ('Npeople', '14')]
12


## Task 5
Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

In [26]:
# all sculptor props
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
   wd:Q1281618 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q98552221'), ('name', 'plastic artist')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5914426'), ('name', 'Template:Infobox artist')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q30059240'), ('name', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q860861'), ('name',

In [27]:
# all sculpture props
queryString = """
SELECT DISTINCT ?p ?pName ?o ?name
WHERE {
   wd:Q860861 ?p ?o.
   ?p sc:name ?pName.
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q223557'), ('name', 'physical object')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6928659'), ('name', 'Category:Sculptures')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P2079'), ('pName', 'fabrication method'), ('o', 'http://www.wikidata.org/entity/Q21711025'), ('name', 'sculpture technique')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q11634'), ('name', 'art of sculpture')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q11634'), ('name', 

In [28]:
# all sculpture subclasses
queryString = """
SELECT DISTINCT ?p ?pName 
WHERE {
   ?p wdt:P279+ wd:Q860861.
   ?p sc:name ?pName.
}
ORDER BY ?pName
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q106766811'), ('pName', 'A Way of Organising a Pig')]
[('p', 'http://www.wikidata.org/entity/Q420816'), ('pName', 'Acrolith')]
[('p', 'http://www.wikidata.org/entity/Q402996'), ('pName', 'Ahu in Easter Island')]
[('p', 'http://www.wikidata.org/entity/Q17157056'), ('pName', 'Amiibo')]
[('p', 'http://www.wikidata.org/entity/Q57709264'), ('pName', 'An hour in the night by Joseph-Michel-Ange Pollet')]
[('p', 'http://www.wikidata.org/entity/Q65679998'), ('pName', 'Ancient Egyptian aegis')]
[('p', 'http://www.wikidata.org/entity/Q10332576'), ('pName', 'Ancient Greek mosaics')]
[('p', 'http://www.wikidata.org/entity/Q4764938'), ('pName', 'Animal figurines')]
[('p', 'http://www.wikidata.org/entity/Q574422'), ('pName', 'Anthropomorphic wooden cult figurines of Central and Northern Europe')]
[('p', 'http://www.wikidata.org/entity/Q63098560'), ('pName', 'Aphrodite in ancient Greek sculptures')]
[('p', 'http://www.wikidata.org/entity/Q98400164'), ('pN

In [29]:
# instances of sculpture subclasses 
queryString = """
SELECT DISTINCT ?p ?pName 
WHERE {
    ?p wdt:P279+ wd:Q860861.
    ?s wdt:P31 ?p;
        sc:name ?pName.
}
ORDER BY ?pName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q483453'), ('pName', '"Bärenbrunnen" von Franz Barwig dem Jüngeren, Dr.-Karl-Landsteiner-Park, Vienna')]
[('p', 'http://www.wikidata.org/entity/Q179700'), ('pName', '"Dialogue" sculpture in Písnická street')]
[('p', 'http://www.wikidata.org/entity/Q20437094'), ('pName', '"Eye" vs. "Eye", Part 4 of the series The Fall of a Hair')]
[('p', 'http://www.wikidata.org/entity/Q20437094'), ('pName', '"Fly Democracy"')]
[('p', 'http://www.wikidata.org/entity/Q179700'), ('pName', '"Four Diamonds" statue')]
[('p', 'http://www.wikidata.org/entity/Q1630622'), ('pName', '"Im Hof" Fountain')]
[('p', 'http://www.wikidata.org/entity/Q179700'), ('pName', '"Tram of Happiness" Group Sculpture')]
[('p', 'http://www.wikidata.org/entity/Q20437094'), ('pName', '"Untitled" (Placebo)')]
[('p', 'http://www.wikidata.org/entity/Q20437094'), ('pName', '"Untitled" (Portrait of Elaine Dannheisser)')]
[('p', 'http://www.wikidata.org/entity/Q20437094'), ('pName', '"Untitled

In [30]:
# instance of sculpture props
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {
    {wd:Q1281618 wdt:P1056 ?p.} #UNION {?p wdt:P279+ wd:Q860861.} 
    ?sc wdt:P31 ?p.
    ?sc ?pr ?o.
    ?pr sc:name ?name.
}
ORDER BY DESC(?name)
LIMIT 800
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P6141'), ('name', 'À nos grands hommes ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P282'), ('name', 'writing system')]
[('pr', 'http://www.wikidata.org/prop/direct/P2031'), ('name', 'work period (start)')]
[('pr', 'http://www.wikidata.org/prop/direct/P937'), ('name', 'work location')]
[('pr', 'http://www.wikidata.org/prop/direct/P6107'), ('name', 'winterthur-glossar URL')]
[('pr', 'http://www.wikidata.org/prop/direct/P5252'), ('name', 'winter view')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P2049'), ('name', 'width')]
[('pr', 'http://www.wikidata.org/prop/direct/P2846'), ('name', 'wheelchair accessibility')]
[('pr', 'http://www.wikidata.org/prop/direct/P4238'), ('name', 'webcam page URL')]
[('pr', 'http://www.wikidata.org/prop/direct/P2872'), ('name', 'visitor centre')]
[('pr', 'http://www.wikidata.org/prop/direct/P8517'), ('name', 'view')]
[('pr', '

In [31]:
#final
queryString = """
SELECT DISTINCT ?sName ?country ?cName
WHERE {
    {wd:Q1281618 wdt:P1056 ?p.} UNION {?p wdt:P279+ wd:Q860861.} 
    VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
    ?mov sc:name ?name.
    ?scul wdt:P31 ?p;
        sc:name ?sName;
        wdt:P495|wdt:P17 ?country;
        wdt:P135 ?mov.
    ?country sc:name ?cName
}
GROUP BY ?name
ORDER BY ?name ?sName
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Allegory of night'), ('country', 'http://www.wikidata.org/entity/Q213'), ('cName', 'Czech Republic')]
[('sName', 'Apollo and Daphne'), ('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy')]
[('sName', 'Bust of Camilla Barbadori'), ('country', 'http://www.wikidata.org/entity/Q35'), ('cName', 'Denmark')]
[('sName', 'Bust of Cardinal Richelieu'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('sName', 'Bust of Carlo Antonio del Pozzo'), ('country', 'http://www.wikidata.org/entity/Q145'), ('cName', 'United Kingdom')]
[('sName', 'Bust of Francesco Barberini'), ('country', 'http://www.wikidata.org/entity/Q30'), ('cName', 'United States of America')]
[('sName', 'Bust of Francesco Barberini'), ('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy')]
[('sName', 'Bust of Hercules'), ('country', 'http://www.wikidata.org/entity/Q31'), ('cName', 'Belgium')]
[('sName', 'Bust of Louis XIV'), ('country', 'http://www.wikidata

In [32]:
#sculptures per movement with country/country of origin Italy
queryString = """
SELECT DISTINCT ?mov ?name ?sName
WHERE {
    {wd:Q1281618 wdt:P1056 ?p.} UNION {?p wdt:P279+ wd:Q860861.} 
    VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
    ?mov sc:name ?name.
    ?scul wdt:P31 ?p;
        wdt:P495|wdt:P17 wd:Q38;
        sc:name ?sName;
        wdt:P135 ?mov.
}
GROUP BY ?mov ?name
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Triton Fountain')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Ecstasy of Saint Teresa')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Memorial to Ippolito Merenda')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Apollo and Daphne')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Memorial to Alessandro Valtrini')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Fontana delle Api')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'The Rape of Proserpina')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Four Rivers Fountain')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Madonnina')]
[('mov', 'http://www.wikida

In [33]:
#sculptures per movement with country/country of origin Italy
queryString = """
SELECT DISTINCT ?mov ?name ?sName
WHERE {
    {wd:Q1281618 wdt:P1056 ?p.} UNION {?p wdt:P279+ wd:Q860861.} 
    VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
    ?mov sc:name ?name.
    ?scul wdt:P31 ?p;
        wdt:P495|wdt:P17 wd:Q38;
        sc:name ?sName;
        wdt:P135 ?mov.
}
GROUP BY ?mov ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Triton Fountain')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Ecstasy of Saint Teresa')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Memorial to Ippolito Merenda')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Apollo and Daphne')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Memorial to Alessandro Valtrini')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Fontana delle Api')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'The Rape of Proserpina')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Four Rivers Fountain')]
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('sName', 'Madonnina')]
[('mov', 'http://www.wikida

Final query for this task

In [34]:
#final
queryString = """
SELECT DISTINCT ?mov ?name (COUNT(DISTINCT ?scul) AS ?Nsculptures)
WHERE {
    {wd:Q1281618 wdt:P1056 ?p.} UNION {?p wdt:P279+ wd:Q860861.} 
    VALUES ?mov {wd:Q37068 wd:Q4692 wd:Q37853}
    ?mov sc:name ?name.
    ?scul wdt:P31 ?p;
        wdt:P495|wdt:P17 wd:Q38;
        wdt:P135 ?mov.
}
GROUP BY ?mov ?name
ORDER BY DESC(?Nsculptures)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('mov', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'), ('Nsculptures', '12')]
1


## Task 6
For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

In [35]:
#final
queryString = """
SELECT DISTINCT ?c ?name
WHERE {
    ?s wdt:P30 ?c.
    ?c sc:name ?name
}

ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('c', 'http://www.wikidata.org/entity/Q107616725'), ('name', 'Alfrey')]
[('c', 'http://www.wikidata.org/entity/Q828'), ('name', 'Americas')]
[('c', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('c', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('c', 'http://www.wikidata.org/entity/Q726891'), ('name', 'Asia')]
[('c', 'http://www.wikidata.org/entity/Q230634'), ('name', 'Asia')]
[('c', 'http://www.wikidata.org/entity/Q97'), ('name', 'Atlantic Ocean')]
[('c', 'http://www.wikidata.org/entity/Q3960'), ('name', 'Australia')]
[('c', 'http://www.wikidata.org/entity/Q620634'), ('name', 'Bir Tawil')]
[('c', 'http://www.wikidata.org/entity/Q664609'), ('name', 'Caribbean')]
[('c', 'http://www.wikidata.org/entity/Q27611'), ('name', 'Central America')]
[('c', 'http://www.wikidata.org/entity/Q10944852'), ('name', 'Essos')]
[('c', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('c'

In [36]:
#props of italy
queryString = """
SELECT DISTINCT ?c ?name
WHERE {
    wd:Q38 wdt:P31 ?c.
    ?c sc:name ?name
}

ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
[('c', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
2


In [37]:
#final
queryString = """
SELECT DISTINCT ?c ?name ?pname#(COUNT(?p) AS ?Npoets)
WHERE {
    ?c #wdt:P30 wd:Q46;     #c in europe
        wdt:P31 wd:Q6256;
        sc:name ?name.
    ?poet wdt:P106 wd:Q49757; #a poet
        wdt:P27|wdt:P19 ?c; 
        wdt:P135 wd:Q37068; #romanticism
        sc:name ?pname.
}
GROUP BY ?c ?name
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('pname', 'Esteban Echeverría')]
[('c', 'http://www.wikidata.org/entity/Q750'), ('name', 'Bolivia'), ('pname', 'María Josefa Mujía')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Manuel de Araújo Porto-Alegre')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Junqueira Freire')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Castro Alves')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Laurindo Rabelo')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Gonçalves de Magalhães, Viscount of Araguaia')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Tobias Barreto')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('pname', 'Bernardo Guimarães')]
[('c', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'),

In [38]:
#final
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
    wd:Q37562 wdt:P27 ?o.
    ?o ?p ?obj.
    ?p sc:name ?pName.
    ?obj sc:name ?oName
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pName', 'basic form of government'), ('obj', 'http://www.wikidata.org/entity/Q123432'), ('oName', 'aristocracy')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q3281534'), ('oName', 'modern history')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q148540'), ('oName', 'Republic of Florence')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pName', 'capital'), ('obj', 'http://www.wikidata.org/entity/Q2044'), ('oName', 'Florence')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q3024240'), ('oName', 'historical country')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q397'), ('oName', 'Latin')]
[('p', 'http://www.wikidata.org/prop/direct/P30')

Final query for this task

In [ ]:
# write your final query

## Task 7
Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query